## Week 5

### Setup

#### Import Libraries

In [ ]:
import matsim
import os
import contextily as ctx
import geopandas as gpd
import matplotlib.pyplot as plt
from plotly.graph_objs import Scattermapbox, Layout, Figure
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import pyproj

from collections import defaultdict

In [ ]:
data_folder = '/cluster/home/salathem/Programming/data'
scenario_path = data_folder + '/scenarios/Zurich/10pct'

### Network

In [ ]:
# This reads in the network in a structure that contains two tables, one for links and one for nodes.
network = matsim.read_network(os.path.join(scenario_path, "zurich_network.xml.gz"))

# this creates a geographic dataframe
network_geo = network.as_geo('epsg:2056')
network_geo

### Population

In [ ]:
plans = matsim.plan_reader(os.path.join(scenario_path, "zurich_population_10pct.xml.gz"))
first_person, first_plan = next(plans)

In [ ]:
# example of how to create a dataframe with activities
plans = matsim.plan_reader(os.path.join(scenario_path, "zurich_population_10pct.xml.gz"))
records = []
for person, plan in plans:
    person_id = person.attrib['id']
    for plan_element in plan:
        if plan_element.tag == 'activity':
            records.append({
                'id': person_id,
                'x': float(plan_element.attrib['x']),
                'y': float(plan_element.attrib['y']),
                'type': plan_element.attrib['type']
            })
activities = pd.DataFrame.from_records(records)
activities

In [ ]:
activities

In [ ]:
# Set up the coordinate transformation.
project = pyproj.Transformer.from_crs(
    "EPSG:2056",  # Source coordinate system (Swiss reference system)
    "EPSG:4326",  # Target coordinate system (global latitude/longitude)
    always_xy=True # The input coordinates are in the form (x, y)
)

# Transform the coordinates and add them to the dataframe.
activities['longitude'], activities['latitude'] = zip(*activities.apply(lambda row: project.transform(row['x'], row['y']), axis=1))

# Now, we'll use Plotly to create a scatter plot on a map.
mapbox_access_token = "your_mapbox_access_token"  # You need a Mapbox access token for the map.

# Create the map figure, using latitude and longitude for positioning.
fig = px.scatter_mapbox(
    activities,
    lat='latitude',
    lon='longitude',
    color='type',  # Assigning color based on the 'type' column.
    hover_name='id',  # Data to be displayed on hover.
    size_max=15,
    zoom=10  # Initial zoom level
)

fig.update_layout(
    mapbox_style="open-street-map",  # Set the map style to "open-street-map".
    mapbox_accesstoken=mapbox_access_token,  # Input your personal Mapbox access token.
    margin={"r": 0, "t": 0, "l": 0, "b": 0}  # Margins of the map
)

fig.show()

### Events